In [1]:
import numpy as np
from scipy.optimize import root
import matplotlib.pyplot as plt
from jupyterthemes import jtplot
jtplot.style()

In [2]:
import quadpy

In [170]:
scheme = quadpy.triangle.hammer_marlowe_stroud_2()

In [171]:
scheme.points

array([[0.66666667, 0.16666667, 0.16666667],
       [0.16666667, 0.66666667, 0.16666667],
       [0.16666667, 0.16666667, 0.66666667]])

In [172]:
scheme.weights

array([0.33333333, 0.33333333, 0.33333333])

In [29]:
from sympy import symbols, simplify, diff, init_printing, integrate, solve, lambdify
init_printing()

In [12]:
xi, eta = symbols("xi, eta", real=True)

In [146]:
N = [(1+1.e-19*xi), xi, eta, xi*eta, xi**2, eta**2]

In [147]:
integrals = [integrate(integrate(N[i],(xi, 0, 1-eta)), (eta, 0, 1)) for i in range(len(N))]
numIntegrals = np.array(integrals, float)

#### Quadrature rule 1: (symmetry arguments)
Let ($\xi_i$, $\eta_i$) be the coordinates of the gauss points and $\omega_j$ be the weights.

In [148]:
numIntegrals

array([0.5       , 0.16666667, 0.16666667, 0.04166667, 0.08333333,
       0.08333333])

In [149]:
Nlam = [lambdify((xi, eta), N[i], "numpy") for i in range(len(N))]

In [224]:
def getPointsWeights(z):
    Xi, Eta, W = z[:2], z[2:4], z[4:]
    Xi = np.hstack((Xi, Eta[-1]))
    Eta = np.hstack((Eta, Xi[-2]))
    W = np.hstack((W, W[0]))
    arr = []
    for i in range(len(numIntegrals)):
        arr.append(0.5*Nlam[i](Xi, Eta) @ W - numIntegrals[i])
    return arr[:]

In [225]:
# guess = np.array([0.5,0.5,0.5,0.,1,1])
guess = np.random.random(6)

In [226]:
getPointsWeights(guess)

In [227]:
soln = root(getPointsWeights, guess, method="hybr", tol=1.e-8)
soln

    fjac: array([[ 1.11728026e-09, -7.19527370e-01, -6.98762841e-03,
        -1.96043878e-01, -6.64973123e-01, -4.01133460e-02],
       [ 3.08287235e-09, -3.09286375e-01, -5.64989366e-01,
        -4.47518098e-01,  4.46554857e-01,  4.30633565e-01],
       [ 2.80890608e-09, -2.20428584e-01,  6.16944960e-01,
         1.95214925e-01,  1.31167214e-01,  7.17966887e-01],
       [-8.83145620e-09, -1.22318379e-01, -5.14643744e-01,
         8.15705263e-01, -1.15020551e-01,  2.03900044e-01],
       [ 9.75295749e-01,  1.25560031e-01, -4.14797834e-02,
        -5.31134926e-02, -1.26507312e-01,  1.11745987e-01],
       [ 2.20903152e-01, -5.54352266e-01,  1.83134788e-01,
         2.34498111e-01,  5.58534547e-01, -4.93362755e-01]])
     fun: array([ 0.00000000e+00,  4.39648318e-14, -2.53963517e-14, -4.05855904e-14,
       -3.27238237e-14,  1.40304435e-14])
 message: 'The solution converged.'
    nfev: 106
     qtf: array([ 1.85384176e-12,  1.95018339e-12,  3.87491795e-12, -3.88253610e-13,
        7.277

In [228]:
soln.x[:-1]

array([ 5.00000000e-01, -4.34075276e-15,  5.00000000e-01,  5.00000000e-01,
        3.33333333e-01])

#### Problem 2
 $$\int_T (2-x - 2 y + 3 x^2 + 4xy - 2y^2) $$

In [231]:
x1, x2, x3 = 0, 1, 0
y1, y2, y3 = 0, 1/2. , 1

In [232]:
x, y = symbols("x, y", real=True)

In [233]:
x = xi*x1 + eta*x2 + (1-xi-eta)*x3
y = xi*y1 + eta*y2 + (1-xi-eta)*y3

In [237]:
from sympy import Matrix
J = Matrix([[x.diff(xi), x.diff(eta)], [y.diff(xi), y.diff(eta)]])

In [239]:
f = 2 - x - 2*y + 3*x**2 + 4*x*y - 2*y**2

In [249]:
Aexact = integrate(integrate(f*J.det(), (xi, 0, 1-eta)), (eta, 0, 1))

In [250]:
xiv, etav, _ = scheme.points.T
Anum = 0
for i in range(xiv.shape[0]):
    Anum += (f*J.det()).subs([(xi, xiv[i]), (eta, etav[i])])*scheme.weights[i]
simplify(Aexact - 1./2*Anum)